# Assignment #1 - Astronomia de Sistemas Planetários

Arthur Araujo Galdino dos Santos

Disponível em: https://github.com/araujoarthur/psastronomyclass/blob/main/A1/V3/A1V4.ipynb

Porque a inclinação do plano orbital e a longitude do nodo ascendente são iguais as 0 no dado disponibilizado, as coordenadas tem sinal invertido em relação as fornecidas por sistemas de calculo de dados. Por este motivo há um segundo arquivo que usa omega = 180.378 e i ~ 0.0067 que pode ser encontrado [aqui](https://github.com/araujoarthur/psastronomyclass/blob/main/A1/V3/A1V4EarthModifiedANL-INCL.ipynb)

### Tabela de Elementos Orbitais e Posições

![Tabela](StandaloneDataTable.png)

### Plot Polar:

![Plot](COMPUTEDDATAPLOTPOL.png)

In [36]:
using DataFrames, PlotlyJS, Statistics
const au = 149597870700.0
;

### Utility Functions

In [37]:
"""
arcCorrection(::Float64) : Float64

Receives an arc and return it if smaller than or equal to 360°, else returns the ARC % 360 in degrees.

[Suboptimal. Won't solve problems like ARC < 0. But works for this assignment.]
"""
function arcCorrection(ARC::Float64)
    if ARC >= 360.0
        return mod(ARC,360.0)
    else
        return Float64(ARC)
    end
end
"""
tt(::Integer, ::Integer, ::Integer, ::Integer, ::Integer) : Float64

Receives a day value, a month value, an year value, an hour value and a minute value and outputs it in days.
"""
function tt(d::Integer, m::Integer, y::Integer, h::Integer, mm::Integer) # \isanst
    return 367 * y - floor(7*(y + ((m+9)/12) )/4) + floor(275*m/9) + d - 730530 + h/24 + mm/1440
end

"""
E(::Float64, ::Float64, [::Float64]) : Float64

Receives the mean anomaly, the eccentricity and optionally [for recursive use] the Ẽ value, all in degrees.
Outputs an eccentric anomaly value in degrees.
"""
function E(M::Float64, EC::Float64, Ẽ::Float64=M)
    M_rad = deg2rad(M)
    Ẽ_rad = deg2rad(Ẽ)

    ΔE_rad = ((M_rad - Ẽ_rad) + EC * sin(Ẽ_rad))/(1 - (EC * cos(Ẽ_rad))) 
    E_deg = rad2deg(Ẽ_rad + ΔE_rad)

    if abs(ΔE_rad) > 5E-6
        return E(M, EC, E_deg)
    else
        return E_deg
    end
end

"""
ν(::Float64, ::Float64) : Float64

Receives the eccentricity and the eccentric anomaly in degrees and return the true anomaly in degrees.
"""
function ν(EC::Float64, E::Float64)
    return 2 * atand(sqrt((1 + EC)/(1 - EC))*tand(E/2))
end

"""
polarReferentialFix(::Float64) : Float64

Receives an angle and return it's equivalent counter clock-wise.
[Not optimal, won't solve problems like congruent angles (<360), but works for this assignment]
"""
function polarReferentialFix(ang::Float64)
    if 0 > ang
        return 360 + ang
    else
        return ang
    end
end

"""
sunDistance(::Float64, ::Float64, ::Float64) : Float64

Receives Major SemiAxis, Eccentricity and True Anomaly (the former in metters, the others in degrees) to output the distance between the sun and the body.
"""
function sunDistance(MSA::Float64, EC::Float64, ν::Float64)
    return (MSA * (1 - EC^2))/(1 + EC * cosd(ν)) 
end


"""
cartesian_X(::Float64, ::Float64, ::Float64, ::Float64) : Float64

Receives the distance between the Sun and the given body (R), the longitude of the ascending node (Ω), the perihelion argument (ω), \n
the true anomaly (v) and the tilt relative to the ecliptic (i) and outputs the rectangular coordinate X.
"""
function cartesian_X(R::Float64, Ω::Float64, ω::Float64, v::Float64, i::Float64)
    return (R * ( ( cosd(Ω) * (cosd(ω + v)) ) - ( sind(Ω) * (sind(ω + v)) * cosd(i) ) ))
end

"""
cartesian_Y(::Float64, ::Float64, ::Float64, ::Float64) : Float64

Receives the distance between the Sun and the given body (R), the longitude of the ascending node (Ω), the perihelion argument (ω), \n
the true anomaly (ν) and the tilt relative to the ecliptic (i) and outputs the rectangular coordinate Y.
"""
function cartesian_Y(R::Float64, Ω::Float64, ω::Float64, ν::Float64, i::Float64)
    return R * ((sind(Ω) * cosd(ω + ν)) + (cosd(Ω) * sind(ω + ν) * cosd(i)))
end

"""
cartesian_Z(::Float64, ::Float64, ::Float64, ::Float64)

Receives the distance between the Sun and the given body (R), the perihelion argument (ω), the true anomaly (ν) and the ∇
tilt relative to the ecliptic (i) and outputs the rectangular coordinate Z.
"""
function cartesian_Z(R::Float64, ω::Float64, ν::Float64, i::Float64)
    return R * sind(ω + ν) * sind(i)
end

"""
ℓ(::Float64, ::Float64) : Float64

Receives the X and Y coordinates and outputs the ecliptic longitude ℓ in degrees (polar coordinates centered on the Sun) based on y sign.
"""
function ℓ(XX::Float64, YY::Float64)
   return atand(YY, XX) 
end

"""
Receives the X, Y and X coordinates and outputs the ecliptic latitude b in degrees.
"""
function b(XX::Float64, YY::Float64, ZZ::Float64)
    return atand((ZZ)/(sqrt((XX^2) + (YY^2))))
end



b

In [38]:
function Mercury(t)
    mercuryDict = Dict()
    mercuryDict["PLANET"] = "Mercury"
    mercuryDict["Ω"] = 48.3313 + 3.24587E-5 * t
    mercuryDict["i"] = 7.0047 + 5.00E-8 * t
    mercuryDict["ω"] = 29.1241 + 1.01444E-5 * t
    mercuryDict["a"] = 0.387098*au
    mercuryDict["e"] = 0.205635 + 5.59E-10 * t
    mercuryDict["M"] = arcCorrection(168.6562 + 4.0923344368 * t)
    mercuryDict["E"] = arcCorrection(E(mercuryDict["M"], mercuryDict["e"]))
    mercuryDict["ν"] = polarReferentialFix(ν(mercuryDict["e"], mercuryDict["E"]))
    mercuryDict["Dist_Sun"] = sunDistance(mercuryDict["a"], mercuryDict["e"], mercuryDict["ν"])
    mercuryDict["X_COORD"] = cartesian_X(mercuryDict["Dist_Sun"], mercuryDict["Ω"], mercuryDict["ω"], mercuryDict["ν"], mercuryDict["i"])
    mercuryDict["Y_COORD"] = cartesian_Y(mercuryDict["Dist_Sun"], mercuryDict["Ω"], mercuryDict["ω"], mercuryDict["ν"], mercuryDict["i"])
    mercuryDict["Z_COORD"] = cartesian_Z(mercuryDict["Dist_Sun"], mercuryDict["ω"], mercuryDict["ν"], mercuryDict["i"])
    mercuryDict["ℓ"] = ℓ(mercuryDict["X_COORD"], mercuryDict["Y_COORD"])
    mercuryDict["ℓ_corr"] = polarReferentialFix(ℓ(mercuryDict["X_COORD"], mercuryDict["Y_COORD"]))
    mercuryDict["b"] = b(mercuryDict["X_COORD"], mercuryDict["Y_COORD"], mercuryDict["Z_COORD"])

    return mercuryDict
end

function Venus(t)
    venusDict = Dict()
    venusDict["PLANET"] = "Venus"
    venusDict["Ω"] = 76.6799 + 2.46590E-5 * t
    venusDict["i"] = 3.3946 + 2.75E-8 * t
    venusDict["ω"] = 54.8910 + 1.38374E-5 * t
    venusDict["a"] = 0.723330 * au
    venusDict["e"] = 0.006773 - 1.302E-9 * t
    venusDict["M"] = arcCorrection(48.0052 + 1.6021302244 * t)
    venusDict["E"] = arcCorrection(E(venusDict["M"], venusDict["e"]))
    venusDict["ν"] = polarReferentialFix(ν(venusDict["e"], venusDict["E"]))
    venusDict["Dist_Sun"] = sunDistance(venusDict["a"], venusDict["e"], venusDict["ν"])
    venusDict["X_COORD"] = cartesian_X(venusDict["Dist_Sun"], venusDict["Ω"], venusDict["ω"], venusDict["ν"], venusDict["i"])
    venusDict["Y_COORD"] = cartesian_Y(venusDict["Dist_Sun"], venusDict["Ω"], venusDict["ω"], venusDict["ν"], venusDict["i"])
    venusDict["Z_COORD"] = cartesian_Z(venusDict["Dist_Sun"], venusDict["ω"], venusDict["ν"], venusDict["i"])
    venusDict["ℓ"] = ℓ(venusDict["X_COORD"], venusDict["Y_COORD"])
    venusDict["ℓ_corr"] = polarReferentialFix(ℓ(venusDict["X_COORD"], venusDict["Y_COORD"]))
    venusDict["b"] = b(venusDict["X_COORD"], venusDict["Y_COORD"], venusDict["Z_COORD"])

    return venusDict
end

function Earth(t)
    earthDict = Dict()
    earthDict["PLANET"] = "Earth"
    earthDict["Ω"] = 0.0
    earthDict["i"] = 0.0
    earthDict["ω"] = 282.9404 + 4.70935E-5 * t
    earthDict["a"] = au
    earthDict["e"] = 0.016709 - 1.151E-9 * t
    earthDict["M"] = arcCorrection(356.0470 + 0.9856002585 * t)
    earthDict["E"] = arcCorrection(E(earthDict["M"], earthDict["e"]))
    earthDict["ν"] = polarReferentialFix(ν(earthDict["e"], earthDict["E"]))
    earthDict["Dist_Sun"] = sunDistance(earthDict["a"], earthDict["e"], earthDict["ν"])
    earthDict["X_COORD"] = cartesian_X(earthDict["Dist_Sun"], earthDict["Ω"], earthDict["ω"], earthDict["ν"], earthDict["i"])
    earthDict["Y_COORD"] = cartesian_Y(earthDict["Dist_Sun"], earthDict["Ω"], earthDict["ω"], earthDict["ν"], earthDict["i"])
    earthDict["Z_COORD"] = cartesian_Z(earthDict["Dist_Sun"], earthDict["ω"], earthDict["ν"], earthDict["i"])
    earthDict["ℓ"] = ℓ(earthDict["X_COORD"], earthDict["Y_COORD"])
    earthDict["ℓ_corr"] = polarReferentialFix(ℓ(earthDict["X_COORD"], earthDict["Y_COORD"]))
    earthDict["b"] = b(earthDict["X_COORD"], earthDict["Y_COORD"], earthDict["Z_COORD"])

    return earthDict
end

function Mars(t)
    marsDict = Dict()
    marsDict["PLANET"] = "Mars"
    marsDict["Ω"] = 49.5574 + 2.11081E-5 * t
    marsDict["i"] = 1.8497 - 1.78E-8 * t
    marsDict["ω"] = 286.5016 + 2.92961E-5 * t
    marsDict["a"] = 1.523688 * au
    marsDict["e"] = 0.093405 + 2.516E-9 * t
    marsDict["M"] = arcCorrection(18.6021 + 0.5240207766 * t)
    marsDict["E"] = arcCorrection(E(marsDict["M"], marsDict["e"]))
    marsDict["ν"] = polarReferentialFix(ν(marsDict["e"], marsDict["E"]))
    marsDict["Dist_Sun"] = sunDistance(marsDict["a"], marsDict["e"], marsDict["ν"])
    marsDict["X_COORD"] = cartesian_X(marsDict["Dist_Sun"], marsDict["Ω"], marsDict["ω"], marsDict["ν"], marsDict["i"])
    marsDict["Y_COORD"] = cartesian_Y(marsDict["Dist_Sun"], marsDict["Ω"], marsDict["ω"], marsDict["ν"], marsDict["i"])
    marsDict["Z_COORD"] = cartesian_Z(marsDict["Dist_Sun"], marsDict["ω"], marsDict["ν"], marsDict["i"])
    marsDict["ℓ"] = ℓ(marsDict["X_COORD"], marsDict["Y_COORD"])
    marsDict["ℓ_corr"] = polarReferentialFix(ℓ(marsDict["X_COORD"], marsDict["Y_COORD"]))
    marsDict["b"] = b(marsDict["X_COORD"], marsDict["Y_COORD"], marsDict["Z_COORD"])

    return marsDict
end

function Jupiter(t)
    jupyterDict = Dict()
    jupyterDict["PLANET"] = "Jupiter"
    jupyterDict["Ω"] = 100.4542 + 2.76854E-5 * t
    jupyterDict["i"] = 1.3030 - 1.557E-7 * t
    jupyterDict["ω"] = 273.8777 + 1.64505E-5 * t
    jupyterDict["a"] = 5.20256 * au
    jupyterDict["e"] = 0.048498 + 4.469E-9 * t
    jupyterDict["M"] = arcCorrection(19.8950 + 0.0830853001 * t)
    jupyterDict["E"] = arcCorrection(E(jupyterDict["M"], jupyterDict["e"]))
    jupyterDict["ν"] = polarReferentialFix(ν(jupyterDict["e"], jupyterDict["E"]))
    jupyterDict["Dist_Sun"] = sunDistance(jupyterDict["a"], jupyterDict["e"], jupyterDict["ν"])
    jupyterDict["X_COORD"] = cartesian_X(jupyterDict["Dist_Sun"], jupyterDict["Ω"], jupyterDict["ω"], jupyterDict["ν"], jupyterDict["i"])
    jupyterDict["Y_COORD"] = cartesian_Y(jupyterDict["Dist_Sun"], jupyterDict["Ω"], jupyterDict["ω"], jupyterDict["ν"], jupyterDict["i"])
    jupyterDict["Z_COORD"] = cartesian_Z(jupyterDict["Dist_Sun"], jupyterDict["ω"], jupyterDict["ν"], jupyterDict["i"])
    jupyterDict["ℓ"] = ℓ(jupyterDict["X_COORD"], jupyterDict["Y_COORD"])
    jupyterDict["ℓ_corr"] = polarReferentialFix(ℓ(jupyterDict["X_COORD"], jupyterDict["Y_COORD"]))
    jupyterDict["b"] = b(jupyterDict["X_COORD"], jupyterDict["Y_COORD"], jupyterDict["Z_COORD"])
    

    return jupyterDict
end

function Saturn(t)
    saturnDict = Dict()
    saturnDict["PLANET"] = "Saturn"
    saturnDict["Ω"] = 113.6634 + 2.38980E-5 * t
    saturnDict["i"] = 2.4886 - 1.081E-7 * t
    saturnDict["ω"] = 339.3939 + 2.97661E-5 * t
    saturnDict["a"] = 9.55475 * au
    saturnDict["e"] = 0.055546 - 9.499E-9 * t
    saturnDict["M"] = arcCorrection(316.9670 + 0.0334442282 * t)
    saturnDict["E"] = arcCorrection(E(saturnDict["M"], saturnDict["e"]))
    saturnDict["ν"] = polarReferentialFix(ν(saturnDict["e"], saturnDict["E"]))
    saturnDict["Dist_Sun"] = sunDistance(saturnDict["a"], saturnDict["e"], saturnDict["ν"])
    saturnDict["X_COORD"] = cartesian_X(saturnDict["Dist_Sun"], saturnDict["Ω"], saturnDict["ω"], saturnDict["ν"], saturnDict["i"])
    saturnDict["Y_COORD"] = cartesian_Y(saturnDict["Dist_Sun"], saturnDict["Ω"], saturnDict["ω"], saturnDict["ν"], saturnDict["i"])
    saturnDict["Z_COORD"] = cartesian_Z(saturnDict["Dist_Sun"], saturnDict["ω"], saturnDict["ν"], saturnDict["i"])
    saturnDict["ℓ"] = ℓ(saturnDict["X_COORD"], saturnDict["Y_COORD"])
    saturnDict["ℓ_corr"] = polarReferentialFix(ℓ(saturnDict["X_COORD"], saturnDict["Y_COORD"]))
    saturnDict["b"] = b(saturnDict["X_COORD"], saturnDict["Y_COORD"], saturnDict["Z_COORD"])


    return saturnDict
end

function Uranus(t)
    uranusDict = Dict()
    uranusDict["PLANET"] = "Uranus"
    uranusDict["Ω"] = 74.0005 + 1.3978E-5 * t
    uranusDict["i"] = 0.7733 + 1.9E-8 * t
    uranusDict["ω"] = 96.6612 + 3.0565E-5 * t
    uranusDict["a"] = (19.18171 - 1.55E-8 * t) * (au)
    uranusDict["e"] = 0.047318 + 7.45E-9 * t
    uranusDict["M"] = arcCorrection(142.5905 + 0.011725806 * t)
    uranusDict["E"] = arcCorrection(E(uranusDict["M"], uranusDict["e"]))
    uranusDict["ν"] = polarReferentialFix(ν(uranusDict["e"], uranusDict["E"]))
    uranusDict["Dist_Sun"] = sunDistance(uranusDict["a"], uranusDict["e"], uranusDict["ν"])
    uranusDict["X_COORD"] = cartesian_X(uranusDict["Dist_Sun"], uranusDict["Ω"], uranusDict["ω"], uranusDict["ν"], uranusDict["i"])
    uranusDict["Y_COORD"] = cartesian_Y(uranusDict["Dist_Sun"], uranusDict["Ω"], uranusDict["ω"], uranusDict["ν"], uranusDict["i"])
    uranusDict["Z_COORD"] = cartesian_Z(uranusDict["Dist_Sun"], uranusDict["ω"], uranusDict["ν"], uranusDict["i"])
    uranusDict["ℓ"] = ℓ(uranusDict["X_COORD"], uranusDict["Y_COORD"])
    uranusDict["ℓ_corr"] = polarReferentialFix(ℓ(uranusDict["X_COORD"], uranusDict["Y_COORD"]))
    uranusDict["b"] = b(uranusDict["X_COORD"], uranusDict["Y_COORD"], uranusDict["Z_COORD"])

    return uranusDict
end

function Neptune(t)
    neptuneDict = Dict()
    neptuneDict["PLANET"] = "Neptune"
    neptuneDict["Ω"] = 131.7806 + 3.0173E-5 * t
    neptuneDict["i"] = 1.7700 - 2.55E-7 * t
    neptuneDict["ω"] = 272.8461 - 6.027E-6 * t
    neptuneDict["a"] = (30.05826 + 3.313E-8 * t) * (au)
    neptuneDict["e"] = 0.008606 + 2.15E-9 * t
    neptuneDict["M"] = arcCorrection(260.2471 + 0.005995147 * t)
    neptuneDict["E"] = arcCorrection(E(neptuneDict["M"], neptuneDict["e"]))
    neptuneDict["ν"] = polarReferentialFix(ν(neptuneDict["e"], neptuneDict["E"]))
    neptuneDict["Dist_Sun"] = sunDistance(neptuneDict["a"], neptuneDict["e"], neptuneDict["ν"])
    neptuneDict["X_COORD"] = cartesian_X(neptuneDict["Dist_Sun"], neptuneDict["Ω"], neptuneDict["ω"], neptuneDict["ν"], neptuneDict["i"])
    neptuneDict["Y_COORD"] = cartesian_Y(neptuneDict["Dist_Sun"], neptuneDict["Ω"], neptuneDict["ω"], neptuneDict["ν"], neptuneDict["i"])
    neptuneDict["Z_COORD"] = cartesian_Z(neptuneDict["Dist_Sun"], neptuneDict["ω"], neptuneDict["ν"], neptuneDict["i"])
    neptuneDict["ℓ"] = ℓ(neptuneDict["X_COORD"], neptuneDict["Y_COORD"])
    neptuneDict["ℓ_corr"] = polarReferentialFix(ℓ(neptuneDict["X_COORD"], neptuneDict["Y_COORD"]))
    neptuneDict["b"] = b(neptuneDict["X_COORD"], neptuneDict["Y_COORD"], neptuneDict["Z_COORD"])

    return neptuneDict
end

Neptune (generic function with 1 method)

In [39]:
gt = tt(1, 7, 2053, 0, 35)

19540.024305555555

In [40]:
iteratorBodies = [Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune]

dfBodies = DataFrame(Mercury(gt))

for iterableBody ∈ iteratorBodies
    push!(dfBodies, iterableBody(gt))
end

dfBodiesCopy = copy(dfBodies)
show(dfBodies, allcols=true)

8×16 DataFrame
 Row │ Dist_Sun    E         M         PLANET   X_COORD      Y_COORD      Z_COORD      a           b          e           i         Ω         ν         ω         ℓ          ℓ_corr   
     │ Float64     Float64   Float64   String   Float64      Float64      Float64      Float64     Float64    Float64     Float64   Float64   Float64   Float64   Float64    Float64  
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ 6.84698e10  207.525   212.971   Mercury   1.24997e10  -6.69984e10  -6.56379e9   5.7909e10   -5.50105   0.205646    7.00568    48.9655  202.489    29.3223   -79.432   280.568
   2 │ 1.07602e11   33.8843   33.6687  Venus    -1.04412e11   2.5217e10    6.37187e9   1.08209e11   3.39486   0.00674756  3.39514    77.1617   34.1004   55.1614   166.422   166.422
   3 │ 1.52084e11  174.787   174.7     Earth    -2.30926e10   1.5032e11   

### Plotting

In [41]:
graphData = DataFrame()
graphData[!, "RADIUS"] = [i for i in 1:8]
graphData[!, "PLANET"] = dfBodies.PLANET
graphData[!, "COORD"] = dfBodies.ℓ

show(graphData)


8×3 DataFrame
 Row │ RADIUS  PLANET   COORD     
     │ Int64   String   Float64   
─────┼────────────────────────────
   1 │      1  Mercury   -79.432
   2 │      2  Venus     166.422
   3 │      3  Earth      98.7336
   4 │      4  Mars      155.37
   5 │      5  Jupiter  -143.508
   6 │      6  Saturn    -21.2865
   7 │      7  Uranus   -175.581
   8 │      8  Neptune    62.7991

In [77]:
plot(scatterpolar(graphData, r=:RADIUS, theta=:COORD, color=:PLANET, mode="markers"), Layout(title="Computed Data Plot", width=600, height=600))

data: [
  "scatterpolar with fields color, mode, r, theta, and type"
]

layout: "layout with fields height, margin, template, title, and width"

# Data from NASA's SPICE Software
http://spice.esac.esa.int/webgeocalc/#OrbitalElements For Orbital Elements. <br>

Kernel: (all GENERIC)<br>
Orbiting Object: [Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune]<br>
Center of Motion: Sun<br>
Reference Frame:ECLIPJ2000<br>
Time System: UTC<br>
Time Format: Calendar date and time<br>
Input Times: Single Time<br>
Time: 2053-07-01T00:00:00.00 // Easier Calc<br>
(True Anomaly and Sun Distance from [NASA Horizons](https://ssd.jpl.nasa.gov/horizons/app.html))

http://spice.esac.esa.int/webgeocalc/#StateVector For Position<br>

Kernel: (all GENERIC)<br>
Target: [Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, Neptune]<br>
OBSERVER: Sun<br>
Frame Name:ECLIPJ2000<br>
Time System: UTC<br>
Time Format: Calendar date and time<br>
Input Times: Single Time<br>
Time: 2053-07-01T00:35:00.00<br>

In [43]:
# Subsetting my DataFrame

subsetBodies = dfBodies[:, [:M, :PLANET, :a, :e, :Ω, :ν, :ω]]
subsetPositions = dfBodies[:, [:PLANET, :X_COORD, :Y_COORD, :Z_COORD, :ℓ, :ℓ_corr, :b]]
rename!(subsetPositions, :X_COORD => :X)
rename!(subsetPositions, :Y_COORD => :Y)
rename!(subsetPositions, :Z_COORD => :Z)
show(subsetBodies, allcols=true)
println("\n\n")
show(subsetPositions, allcols=true)

8×7 DataFrame
 Row │ M         PLANET   a           e           Ω         ν         ω        
     │ Float64   String   Float64     Float64     Float64   Float64   Float64  
─────┼─────────────────────────────────────────────────────────────────────────
   1 │ 212.971   Mercury  5.7909e10   0.205646     48.9655  202.489    29.3223
   2 │  33.6687  Venus    1.08209e11  0.00674756   77.1617   34.1004   55.1614
   3 │ 174.7     Earth    1.49598e11  0.0166865     0.0     174.873   283.861
   4 │ 177.981   Mars     2.2794e11   0.0934542    49.9699  178.319   287.074
   5 │ 203.384   Jupiter  7.78292e11  0.0485853   100.995   201.291   274.199
   6 │ 250.468   Saturn   1.42937e12  0.0553604   114.13    244.635   339.976
   7 │  11.713   Uranus   2.8695e12   0.0474636    74.2736   12.885    97.2584
   8 │  17.3924  Neptune  4.49675e12  0.00864801  132.37     17.6917  272.728


8×7 DataFrame
 Row │ PLANET   X            Y            Z            ℓ          ℓ_corr    b         
     │ String   

In [44]:
MercurySPICE = Dict(
    "PLANET" => "Mercury",
    "Ω" => 48.26394929,
    "ω" => 29.27515022,
    "a" => 68126020.15398264E3, #Convert from KM
    "e" => 0.20565023,
    "M" => 216.96933818,
    "i" => 7.00184025,
    "X" => 14911091.80597833E3,
    "Y" => -66118957.10648166E3,
    "Z" => -6772425.47931979E3,
    "ℓ" => -77.29130718,
    "ℓ_corr" => polarReferentialFix(-77.29130718),
    "b" => -5.70597146,
    "ν" => 205.2995670038993,
    "R" => 6.778207702727926E10
)

VenusSPICE = Dict(
    "PLANET" => "Venus",
    "Ω" => 76.53136424,
    "ω" => 55.23916999,
    "a" => 107611084.49418102E3,
    "e" => 	0.00676238,
    "M" => 35.04145354,
    "i" => 3.39427835,
    "X" => -104794789.27783182E3,
    "Y" => 23615111.32284366E3,
    "Z" => 6370740.54597109E3,
    "ℓ" => 167.30073594,
    "ℓ_corr" => 167.30073594,
    "b" => 3.39397305,
    "ν" => 35.50022680504404,
    "R" => 1.074549503179932E11
)

EarthSPICE = Dict(
    "PLANET" => "Earth",
    "Ω" => 180.37840376,
    "ω" => 282.44577374,
    "a" => 152082619.11011820E3,
    "e" => 0.01751289,
    "M" => 175.94943770,
    "i" => 0.00672579,
    "X" => 23561118.31566819E3,
    "Y" => -150246452.29559687E3,
    "Z" => 17654.89506852E3, # For me there's no reason for earth to be z-displaced in ecliptic's frame
    "ℓ" => -81.06449477,
    "ℓ_corr" => polarReferentialFix(-81.06449477),
    "b" => 0.00665092,
    "ν" => 176.0882921447710,
    "R" => 1.517150540821351E11
)

MarsSPICE = Dict(
    "PLANET" => "Mars",
    "Ω" => 49.39843323,
    "ω" => 286.91935417,
    "a" => 249239379.40676948E3,
    "e" => 0.09351113,
    "M" => 178.47274288,
    "i" => 1.84548902,
    "X" => -225897389.22283340E3,
    "Y" => 105028364.72766489E3,
    "Z" => 7728701.20609896E3,
    "ℓ" => 155.06447561,
    "ℓ_corr" => 155.06447561,
    "b" => 1.77697751,
    "ν" => 178.7280093491614,
    "R" => 2.491828348843164E11
)

JupiterSPICE = Dict(
    "PLANET" => "Jupiter",
    "Ω" => 100.58171490,
    "ω" => 274.90404577,
    "a" => 813294159.49340240E3,
    "e" => 0.04738841,
    "M" => 202.30773215,
    "i" => 1.30343505, 
    "X" => -659099013.85965610E3,
    "Y" => -476188208.77358800E3,
    "Z" => 16731227.13450408E3,
    "ℓ" => -144.15250724,
    "ℓ_corr" => polarReferentialFix(-144.15250724),
    "b" => 1.17878243,
    "ν" => 199.9601671373209,
    "R" => 8.128464215333256E11
    
)

SaturnSPICE = Dict(
    "PLANET" => "Saturn",
    "Ω" => 113.47181865,
    "ω" => 338.79130237,
    "a" => 1453474979.37304350E3,
    "e" => 0.05327919,
    "M" => 251.78553602,
    "i" => 2.48725019,
    "X" => 1350310213.83293800E3,
    "Y" => -535972188.34747136E3,
    "Z" => -44528446.49176982E3,
    "ℓ" => -21.64937666,
    "ℓ_corr" => polarReferentialFix(-21.64937666),
    "b" => -1.75558099,
    "ν" => 245.8333400468297,
    "R" => 1.453559593042406E12
)

UranusSPICE = Dict(
    "PLANET" => "Uranus",
    "Ω" => 73.95336561,
    "ω" => 98.96809742,
    "a" => 2738027824.74679600E3,
    "e" => 0.05082914,
    "M" => 9.66654791,
    "i" => 0.77187598,
    "X" => -2732305452.18072700E3,
    "Y" => -173487143.73558030E3,
    "Z" => 34730880.73453543E3,
    "ℓ" => -176.36689403,
    "ℓ_corr" => polarReferentialFix(-176.36689403),
    "b" => 0.72679545,
    "ν" => 10.71881736629351,
    "R" => 2.737747592415333E12
)

NeptuneSPICE = Dict(
    "PLANET" => "Neptune",
    "Ω" => 132.05175038,
    "ω" => 269.26791696,
    "a" => 4460940793.96299800E3,
    "e" => 0.00585937,
    "M" => 20.48466102,
    "i" => 1.76743947,
    "X" => 2089979961.21707370E3,
    "Y" => 3938941296.74271060E3,
    "Z" => -129298881.91905070E3,
    "ℓ" =>  62.04983143,
    "ℓ_corr" => 62.04983143,
    "b" => -1.66093168,
    "ν" => 20.08143075194107,
    "R" => 4.461423855999846E12
)

dfSPICE = DataFrame(MercurySPICE)
push!(dfSPICE, VenusSPICE)
push!(dfSPICE, EarthSPICE)
push!(dfSPICE, MarsSPICE)
push!(dfSPICE, JupiterSPICE)
push!(dfSPICE, SaturnSPICE)
push!(dfSPICE, UranusSPICE)
push!(dfSPICE, NeptuneSPICE)

dfSPICEcopy = DataFrame(dfSPICE)

subsetBodies = dfBodies[!, [:PLANET, :Ω, :a, :e, :i, :M]]
dfSPICEpos = dfSPICE[!, [:PLANET, :X, :Y, :Z, :ℓ, :ℓ_corr, :b]]
dfSPICE = dfSPICE[!, [:PLANET, :Ω, :a, :e, :i, :M]]
printstyled("\n\nSPICE Data\n", color=:red)
show(dfSPICE, allcols=true)




SPICE Data
8×6 DataFrame
 Row │ PLANET   Ω         a           e           i           M         
     │ String   Float64   Float64     Float64     Float64     Float64   
─────┼──────────────────────────────────────────────────────────────────
   1 │ Mercury   48.2639  6.8126e10   0.20565     7.00184     216.969
   2 │ Venus     76.5314  1.07611e11  0.00676238  3.39428      35.0415
   3 │ Earth    180.378   1.52083e11  0.0175129   0.00672579  175.949
   4 │ Mars      49.3984  2.49239e11  0.0935111   1.84549     178.473
   5 │ Jupiter  100.582   8.13294e11  0.0473884   1.30344     202.308
   6 │ Saturn   113.472   1.45347e12  0.0532792   2.48725     251.786
   7 │ Uranus    73.9534  2.73803e12  0.0508291   0.771876      9.66655
   8 │ Neptune  132.052   4.46094e12  0.00585937  1.76744      20.4847

In [45]:
printstyled("\n\nMy Calculated Data\n", color=:green)
show(subsetBodies, allcols=true, allrows=true)



My Calculated Data
8×6 DataFrame
 Row │ PLANET   Ω         a           e           i         M        
     │ String   Float64   Float64     Float64     Float64   Float64  
─────┼───────────────────────────────────────────────────────────────
   1 │ Mercury   48.9655  5.7909e10   0.205646    7.00568   212.971
   2 │ Venus     77.1617  1.08209e11  0.00674756  3.39514    33.6687
   3 │ Earth      0.0     1.49598e11  0.0166865   0.0       174.7
   4 │ Mars      49.9699  2.2794e11   0.0934542   1.84935   177.981
   5 │ Jupiter  100.995   7.78292e11  0.0485853   1.29996   203.384
   6 │ Saturn   114.13    1.42937e12  0.0553604   2.48649   250.468
   7 │ Uranus    74.2736  2.8695e12   0.0474636   0.773671   11.713
   8 │ Neptune  132.37    4.49675e12  0.00864801  1.76502    17.3924

### Comparsion Between Calculated Orbital Elements and SPICE Generated Orbital Elements

In [46]:
dfDifferenceGENtoSPICE = subsetBodies[!, Not(:PLANET)] .- dfSPICE[!, Not(:PLANET)]
dfRateGENtoSPICE = subsetBodies[!, Not(:PLANET)] ./ dfSPICE[!, Not(:PLANET)]
dfStatisticsRate = describe(dfRateGENtoSPICE)
dfRateGENtoSPICE[!, "PLANET"] = ["Mercury", "Venus", "Earth", "Mars", "Jupiter", "Saturn", "Uranus", "Neptune"]
show(dfRateGENtoSPICE, allcols=true, allrows=true)
println("\n")
show(dfStatisticsRate, allcols=true, allrows=true)

function percentualDifference(n1::Float64, n2::Float64)
    return (n1 - n2)/(n1 + n2)
end

8×6 DataFrame
 Row │ Ω        a         e         i         M         PLANET  
     │ Float64  Float64   Float64   Float64   Float64   String  
─────┼──────────────────────────────────────────────────────────
   1 │ 1.01454  0.850028  0.999979  1.00055   0.98157   Mercury
   2 │ 1.00824  1.00555   0.997808  1.00025   0.960826  Venus
   3 │ 0.0      0.983662  0.952813  0.0       0.992899  Earth
   4 │ 1.01157  0.914544  0.999391  1.00209   0.997244  Mars
   5 │ 1.00411  0.956962  1.02526   0.997332  1.00532   Jupiter
   6 │ 1.0058   0.983416  1.03906   0.999693  0.994767  Saturn
   7 │ 1.00433  1.04802   0.933787  1.00233   1.21171   Uranus
   8 │ 1.00241  1.00803   1.47593   0.99863   0.849046  Neptune

5×7 DataFrame
 Row │ variable  mean      min       median    max      nmissing  eltype   
     │ Symbol    Float64   Float64   Float64   Float64  Int64     DataType 
─────┼─────────────────────────────────────────────────────────────────────
   1 │ Ω         0.881375  0.0       1.00507 

percentualDifference (generic function with 1 method)

In [47]:
percentualDiffDF = percentualDifference.(subsetBodies[!, Not(:PLANET)], dfSPICE[!, Not(:PLANET)])
percentualDiffDF .*= 100

,Ω,a,e,i,M
,Float64,Float64,Float64,Float64,Float64
1,0.721586,-8.10646,-0.00104721,0.0273907,-0.930078
2,0.410151,0.276872,-0.109705,0.0126521,-1.99785
3,-100.0,-0.823636,-2.41636,-100.0,-0.356319
4,0.575052,-4.46349,-0.0304695,0.104556,-0.138007
5,0.205112,-2.19921,1.24713,-0.133573,0.265239
6,0.289342,-0.836144,1.91569,-0.0153299,-0.262319
7,0.216064,2.34453,-3.42402,0.116158,9.57215
8,0.120425,0.399743,19.2222,-0.0685692,-8.16389


Generally, the SPICE values are higher than mine (withot high expression unless in case of Earth's elements)

In [48]:
describe(percentualDiffDF)

,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Int64,DataType
1,Ω,-12.1828,-100.0,0.252703,0.721586,0,Float64
2,a,-1.67598,-8.10646,-0.82989,2.34453,0,Float64
3,e,2.05043,-3.42402,-0.0157583,19.2222,0,Float64
4,i,-12.4946,-100.0,-0.0013389,0.116158,0,Float64
5,M,-0.251384,-8.16389,-0.309319,9.57215,0,Float64


In [49]:
printstyled("My Data (Positions)\n", color=:blue)
show(subsetPositions, allcols=true)
println("\n\n")
printstyled("SPICE Data (Positions)\n", color=:magenta)
show(dfSPICEpos, allcols=true)

My Data (Positions)
8×7 DataFrame
 Row │ PLANET   X            Y            Z            ℓ          ℓ_corr    b         
     │ String   Float64      Float64      Float64      Float64    Float64   Float64   
─────┼────────────────────────────────────────────────────────────────────────────────
   1 │ Mercury   1.24997e10  -6.69984e10  -6.56379e9    -79.432   280.568   -5.50105
   2 │ Venus    -1.04412e11   2.5217e10    6.37187e9    166.422   166.422    3.39486
   3 │ Earth    -2.30926e10   1.5032e11    0.0           98.7336   98.7336   0.0
   4 │ Mars     -2.26446e11   1.03818e11   7.75461e9    155.37    155.37     1.78299
   5 │ Jupiter  -6.53688e11  -4.83554e11   1.66543e10  -143.508   216.492    1.1734
   6 │ Saturn    1.3594e12   -5.29636e11  -4.44706e10   -21.2865  338.714   -1.74593
   7 │ Uranus   -2.72807e12  -2.10813e11   3.4689e10   -175.581   184.419    0.726346
   8 │ Neptune   2.03772e12   3.96482e12  -1.28728e11    62.7991   62.7991  -1.65407


SPICE Data (Positions)
8×7 

In [50]:
positionsDif = subsetPositions[!,Not(:PLANET)] .- dfSPICEpos[!,Not(:PLANET)]
positionsDif[!, "PLANET"] = dfSPICEpos[!, "PLANET"]

positionsDif = positionsDif[:, [:PLANET, :X, :Y, :Z, :ℓ, :b]]

,PLANET,X,Y,Z,ℓ,b
,String,Float64,Float64,Float64,Float64,Float64
1,Mercury,-2.41136e9,-8.79454e8,2.08633e8,-2.14065,0.204918
2,Venus,3.83106e8,1.60185e9,1.12778e6,-0.8785,0.000882178
3,Earth,-4.66537e10,3.00567e11,-1.76549e7,179.798,-0.00665092
4,Mars,-5.48924e8,-1.21013e9,2.59061e7,0.305615,0.00601605
5,Jupiter,5.41129e9,-7.36568e9,-7.69184e7,0.644035,-0.0053869
6,Saturn,9.08513e9,6.33634e9,5.788e7,0.362924,0.00964992
7,Uranus,4.23527e9,-3.73263e10,-4.18327e7,0.785682,-0.000449759
8,Neptune,-5.22637e10,2.58776e10,5.70601e8,0.749298,0.00686008


In [51]:
positionsRatio = subsetPositions[!,Not(:PLANET)] ./ dfSPICEpos[!,Not(:PLANET)]

,X,Y,Z,ℓ,ℓ_corr,b
,Float64,Float64,Float64,Float64,Float64,Float64
1,0.838284,1.0133,0.969194,1.0277,0.992428,0.964087
2,0.996344,1.06783,1.00018,0.994749,0.994749,1.00026
3,-0.980114,-1.00049,0.0,-1.21796,0.353966,0.0
4,1.00243,0.988478,1.00335,1.00197,1.00197,1.00339
5,0.99179,1.01547,0.995403,0.995532,1.00298,0.99543
6,1.00673,0.988178,0.9987,0.983236,1.00107,0.994503
7,0.99845,1.21515,0.998796,0.995545,1.00428,0.999381
8,0.974993,1.00657,0.995587,1.01208,1.01208,0.99587


In [52]:
positionsPercentDiff = percentualDifference.(subsetPositions[!,Not(:PLANET)], dfSPICEpos[!,Not(:PLANET)])
positionsPercentDiff .*= 100

,X,Y,Z,ℓ,ℓ_corr,b
,Float64,Float64,Float64,Float64,Float64,Float64
1,-8.7971,0.66066,-1.56441,1.36588,-0.380036,-1.82848
2,-0.183123,3.28031,0.00885047,-0.263242,-0.263242,0.0129946
3,-9957.51,4.06544e5,-100.0,1017.58,-47.7142,-100.0
4,0.121351,-0.579434,0.167316,0.0984476,0.0984476,0.168992
5,-0.412199,0.767465,-0.230394,-0.223887,0.148965,-0.229017
6,0.335281,-0.594622,-0.0650344,-0.845271,0.0536026,-0.275593
7,-0.0775638,9.71279,-0.0602604,-0.223238,0.21347,-0.0309508
8,-1.26617,0.327409,-0.22114,0.600164,0.600164,-0.20694


## PLotting SPICE Data

In [53]:
dfSPICEposRadius = Dict("ℓ" => dfSPICEpos[!,"ℓ"],
"RADIUS" => [i for i in 1:8])
plot(scatterpolar(dfSPICEpos, r=:RADIUS, theta=:ℓ, mode="markers"))

data: [
  "scatterpolar with fields mode, r, theta, and type"
]

layout: "layout with fields margin and template"

In [54]:
myPositionsIDX = copy(subsetPositions)
myPositionsIDX[!, "PLANET"] .*= "_CALC" #String concatenation in jl is * not +
dfSPICEposIDX = copy(dfSPICEpos)
dfSPICEposIDX[!, "PLANET"] .*= "_SPICE"
;

In [55]:
FullPosDF = DataFrame()
append!(FullPosDF, dfSPICEposIDX)
append!(FullPosDF, myPositionsIDX)
sort!(FullPosDF, [order(:PLANET)])
;

In [56]:
show(dfBodiesCopy, allcols=true)
dfBodiesCopy = dfBodiesCopy[!, [:PLANET, :Ω, :i, :ω, :e, :a, :M, :E, :ν, :Dist_Sun, :X_COORD, :Y_COORD, :Z_COORD, :ℓ, :ℓ_corr, :b]]
rename!(dfBodiesCopy, :Dist_Sun => :R)
rename!(dfBodiesCopy, :X_COORD => :X)
rename!(dfBodiesCopy, :Y_COORD => :Y)
rename!(dfBodiesCopy, :Z_COORD => :Z)
;


8×16 DataFrame
 Row │ Dist_Sun    E         M         PLANET   X_COORD      Y_COORD      Z_COORD      a           b          e           i         Ω         ν         ω         ℓ          ℓ_corr   
     │ Float64     Float64   Float64   String   Float64      Float64      Float64      Float64     Float64    Float64     Float64   Float64   Float64   Float64   Float64    Float64  
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ 6.84698e10  207.525   212.971   Mercury   1.24997e10  -6.69984e10  -6.56379e9   5.7909e10   -5.50105   0.205646    7.00568    48.9655  202.489    29.3223   -79.432   280.568
   2 │ 1.07602e11   33.8843   33.6687  Venus    -1.04412e11   2.5217e10    6.37187e9   1.08209e11   3.39486   0.00674756  3.39514    77.1617   34.1004   55.1614   166.422   166.422
   3 │ 1.52084e11  174.787   174.7     Earth    -2.30926e10   1.5032e11   

In [57]:
dfBodiesWOEcc = DataFrame(dfBodiesCopy[!, Not(:E)])
dfSPICEcopySorted = dfSPICEcopy[:, [:PLANET, :Ω, :i, :ω, :e, :a, :M, :ν, :R, :X, :Y, :Z, :ℓ, :ℓ_corr, :b]]
println("\n\n")
#show(dfSPICEcopySorted, allcols=true)
#show(dfBodiesWOEcc, allcols=true)

# Just making things clear. No time to refactor, sry.
completeCalculated = copy(dfBodiesWOEcc)
completeSPICE = copy(dfSPICEcopySorted)
;

In [58]:
completeSPICE[!, "PLANET"] .*= "_SPICE" # Concatenation in Julia is made by the * operator.
completeCalculated[!, "PLANET"] .*= "_CALC"
;

In [59]:
mergedData = DataFrame()

append!(mergedData, completeSPICE)
append!(mergedData, completeCalculated)

sort!(mergedData, [order(:PLANET)])

show(mergedData, allcols=true)

mergedDataWork = copy(mergedData)
;

16×15 DataFrame
 Row │ PLANET         Ω         i           ω         e           a           M          ν         R           X            Y            Z            ℓ          ℓ_corr    b           
     │ String         Float64   Float64     Float64   Float64     Float64     Float64    Float64   Float64     Float64      Float64      Float64      Float64    Float64   Float64     
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ Earth_CALC       0.0     0.0         283.861   0.0166865   1.49598e11  174.7      174.873   1.52084e11  -2.30926e10   1.5032e11    0.0           98.7336   98.7336   0.0
   2 │ Earth_SPICE    180.378   0.00672579  282.446   0.0175129   1.52083e11  175.949    176.088   1.51715e11   2.35611e10  -1.50246e11   1.76549e7    -81.0645  278.936    0.00665092
   3 │ Jupiter_CALC   100.995   1.29996     274.199   0.0485853   7.78292

In [60]:
show(mergedDataWork, allcols=true)

16×15 DataFrame
 Row │ PLANET         Ω         i           ω         e           a           M          ν         R           X            Y            Z            ℓ          ℓ_corr    b           
     │ String         Float64   Float64     Float64   Float64     Float64     Float64    Float64   Float64     Float64      Float64      Float64      Float64    Float64   Float64     
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ Earth_CALC       0.0     0.0         283.861   0.0166865   1.49598e11  174.7      174.873   1.52084e11  -2.30926e10   1.5032e11    0.0           98.7336   98.7336   0.0
   2 │ Earth_SPICE    180.378   0.00672579  282.446   0.0175129   1.52083e11  175.949    176.088   1.51715e11   2.35611e10  -1.50246e11   1.76549e7    -81.0645  278.936    0.00665092
   3 │ Jupiter_CALC   100.995   1.29996     274.199   0.0485853   7.78292

    Mercury is consistent. Remove from DataFrame.

In [61]:
delete!(mergedDataWork, 7:8)
show(mergedDataWork, allcols=true)

14×15 DataFrame
 Row │ PLANET         Ω         i           ω         e           a           M          ν         R           X            Y            Z            ℓ          ℓ_corr    b           
     │ String         Float64   Float64     Float64   Float64     Float64     Float64    Float64   Float64     Float64      Float64      Float64      Float64    Float64   Float64     
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ Earth_CALC       0.0     0.0         283.861   0.0166865   1.49598e11  174.7      174.873   1.52084e11  -2.30926e10   1.5032e11    0.0           98.7336   98.7336   0.0
   2 │ Earth_SPICE    180.378   0.00672579  282.446   0.0175129   1.52083e11  175.949    176.088   1.51715e11   2.35611e10  -1.50246e11   1.76549e7    -81.0645  278.936    0.00665092
   3 │ Jupiter_CALC   100.995   1.29996     274.199   0.0485853   7.78292

Earth is consistent, deleting it. 

In [62]:
delete!(mergedDataWork, 1:2)
show(mergedDataWork, allcols=true)

12×15 DataFrame
 Row │ PLANET         Ω         i         ω         e           a           M          ν         R           X            Y            Z            ℓ          ℓ_corr    b         
     │ String         Float64   Float64   Float64   Float64     Float64     Float64    Float64   Float64     Float64      Float64      Float64      Float64    Float64   Float64   
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ Jupiter_CALC   100.995   1.29996   274.199   0.0485853   7.78292e11  203.384    201.291   8.13271e11  -6.53688e11  -4.83554e11   1.66543e10  -143.508   216.492    1.1734
   2 │ Jupiter_SPICE  100.582   1.30344   274.904   0.0473884   8.13294e11  202.308    199.96    8.12846e11  -6.59099e11  -4.76188e11   1.67312e10  -144.153   215.847    1.17878
   3 │ Mars_CALC       49.9699  1.84935   287.074   0.0934542   2.2794e11   177.981    17

Neptune is consistent. Deleting it.

In [63]:
delete!(mergedDataWork, 5:6)
show(mergedDataWork, allcols=true)

10×15 DataFrame
 Row │ PLANET         Ω         i         ω         e           a           M          ν         R           X            Y            Z            ℓ          ℓ_corr   b         
     │ String         Float64   Float64   Float64   Float64     Float64     Float64    Float64   Float64     Float64      Float64      Float64      Float64    Float64  Float64   
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ Jupiter_CALC   100.995   1.29996   274.199   0.0485853   7.78292e11  203.384    201.291   8.13271e11  -6.53688e11  -4.83554e11   1.66543e10  -143.508   216.492   1.1734
   2 │ Jupiter_SPICE  100.582   1.30344   274.904   0.0473884   8.13294e11  202.308    199.96    8.12846e11  -6.59099e11  -4.76188e11   1.67312e10  -144.153   215.847   1.17878
   3 │ Mars_CALC       49.9699  1.84935   287.074   0.0934542   2.2794e11   177.981    178.319

Saturn is consistent. Deleting it.

In [64]:
delete!(mergedDataWork, 5:6)
show(mergedDataWork, allcols=true)

8×15 DataFrame
 Row │ PLANET         Ω         i         ω         e           a           M          ν         R           X            Y            Z           ℓ         ℓ_corr   b        
     │ String         Float64   Float64   Float64   Float64     Float64     Float64    Float64   Float64     Float64      Float64      Float64     Float64   Float64  Float64  
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ Jupiter_CALC   100.995   1.29996   274.199   0.0485853   7.78292e11  203.384    201.291   8.13271e11  -6.53688e11  -4.83554e11  1.66543e10  -143.508  216.492  1.1734
   2 │ Jupiter_SPICE  100.582   1.30344   274.904   0.0473884   8.13294e11  202.308    199.96    8.12846e11  -6.59099e11  -4.76188e11  1.67312e10  -144.153  215.847  1.17878
   3 │ Mars_CALC       49.9699  1.84935   287.074   0.0934542   2.2794e11   177.981    178.319   2.49231e11  -

In [65]:
dfBodiesToPlot = DataFrame()
dfBodiesToPlot = copy(dfBodiesCopy[!, [:PLANET,:ℓ, :X, :Y]])
dfBodiesToPlot[!, "SOURCE"] .= 1
dfBodiesToPlot[!,"RADIUS"] = [i for i in 1:8]
dfSPICEPlot = copy(dfSPICEcopy[:, [:PLANET, :ℓ, :X, :Y]])
dfSPICEPlot[!, "SOURCE"] .= 2
dfSPICEPlot[!,"RADIUS"] = [i for i in 1:8]

show(dfSPICEPlot, allcols=true)
println("\n\n")
show(dfBodiesToPlot, allcols=true)

plotFrame = DataFrame()
append!(plotFrame, dfBodiesToPlot)
append!(plotFrame, dfSPICEPlot)


8×6 DataFrame
 Row │ PLANET   ℓ          X            Y            SOURCE  RADIUS 
     │ String   Float64    Float64      Float64      Int64   Int64  
─────┼──────────────────────────────────────────────────────────────
   1 │ Mercury   -77.2913   1.49111e10  -6.6119e10        2       1
   2 │ Venus     167.301   -1.04795e11   2.36151e10       2       2
   3 │ Earth     -81.0645   2.35611e10  -1.50246e11       2       3
   4 │ Mars      155.064   -2.25897e11   1.05028e11       2       4
   5 │ Jupiter  -144.153   -6.59099e11  -4.76188e11       2       5
   6 │ Saturn    -21.6494   1.35031e12  -5.35972e11       2       6
   7 │ Uranus   -176.367   -2.73231e12  -1.73487e11       2       7
   8 │ Neptune    62.0498   2.08998e12   3.93894e12       2       8


8×6 DataFrame
 Row │ PLANET   ℓ          X            Y            SOURCE  RADIUS 
     │ String   Float64    Float64      Float64      Int64   Int64  
─────┼──────────────────────────────────────────────────────────────
   1 │ Mercu

,PLANET,ℓ,X,Y,SOURCE,RADIUS
,String,Float64,Float64,Float64,Int64,Int64
1,Mercury,-79.432,1.24997e10,-6.69984e10,1,1
2,Venus,166.422,-1.04412e11,2.5217e10,1,2
3,Earth,98.7336,-2.30926e10,1.5032e11,1,3
4,Mars,155.37,-2.26446e11,1.03818e11,1,4
5,Jupiter,-143.508,-6.53688e11,-4.83554e11,1,5
6,Saturn,-21.2865,1.3594e12,-5.29636e11,1,6
7,Uranus,-175.581,-2.72807e12,-2.10813e11,1,7
8,Neptune,62.7991,2.03772e12,3.96482e12,1,8
9,Mercury,-77.2913,1.49111e10,-6.6119e10,2,1


In [72]:
plot([scatterpolar(dfSPICEPlot, r=:RADIUS, theta=:ℓ, marker=attr(color=:green), mode="markers", name="SPICE", text=["Mercury", "Venus", "Earth", "Mars", "Jupiter", "Saturn", "Uranus", "Neptune"]), 
      scatterpolar(dfBodiesToPlot, r=:RADIUS, theta=:ℓ, marker=attr(color=:red), mode="markers",  name="CALCULATIONS", text=["Mercury", "Venus", "Earth", "Mars", "Jupiter", "Saturn", "Uranus", "Neptune"])
    ], Layout(title="SPICE Generated and Manually Calculated Data [Polar]"))



    #TODO: Check on variation (SPICE's Lat goes from -90 to +90. SPICE's Lon goes from -180 to +180 (mine is 0;360°))

data: [
  "scatterpolar with fields marker, mode, name, r, text, theta, and type",
  "scatterpolar with fields marker, mode, name, r, text, theta, and type"
]

layout: "layout with fields margin, template, and title"

In [73]:
plot([scatter(dfSPICEPlot, x=:X, y=:Y, marker=attr(color=:green), mode="markers", name="SPICE", text=["Mercury", "Venus", "Earth", "Mars", "Jupiter", "Saturn", "Uranus", "Neptune"]), 
scatter(dfBodiesToPlot, x=:X, y=:Y, marker=attr(color=:red), mode="markers", name="CALCULATION", 
text=["Mercury", "Venus", "Earth", "Mars", "Jupiter", "Saturn", "Uranus", "Neptune"])], Layout(title="SPICE Generated and Manually Calculated Data [RECT]"))

data: [
  "scatter with fields marker, mode, name, text, type, x, and y",
  "scatter with fields marker, mode, name, text, type, x, and y"
]

layout: "layout with fields margin, template, and title"

In [68]:
mergedDataEarthSolve = copy(mergedData)
delete!(mergedDataEarthSolve, 3:16)
;

In [69]:
show(mergedDataEarthSolve, allcols=true)

2×15 DataFrame
 Row │ PLANET       Ω        i           ω        e          a           M        ν        R           X            Y            Z          ℓ         ℓ_corr    b          
     │ String       Float64  Float64     Float64  Float64    Float64     Float64  Float64  Float64     Float64      Float64      Float64    Float64   Float64   Float64    
─────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ Earth_CALC     0.0    0.0         283.861  0.0166865  1.49598e11  174.7    174.873  1.52084e11  -2.30926e10   1.5032e11   0.0         98.7336   98.7336  0.0
   2 │ Earth_SPICE  180.378  0.00672579  282.446  0.0175129  1.52083e11  175.949  176.088  1.51715e11   2.35611e10  -1.50246e11  1.76549e7  -81.0645  278.936   0.00665092

In [82]:
cleanTable = DataFrame(dfBodiesCopy)

show(cleanTable, allcols=true)

8×16 DataFrame
 Row │ PLANET   Ω         i         ω         e           a           M         E         ν         R           X            Y            Z            ℓ          ℓ_corr    b         
     │ String   Float64   Float64   Float64   Float64     Float64     Float64   Float64   Float64   Float64     Float64      Float64      Float64      Float64    Float64   Float64   
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ Mercury   48.9655  7.00568    29.3223  0.205646    5.7909e10   212.971   207.525   202.489   6.84698e10   1.24997e10  -6.69984e10  -6.56379e9    -79.432   280.568   -5.50105
   2 │ Venus     77.1617  3.39514    55.1614  0.00674756  1.08209e11   33.6687   33.8843   34.1004  1.07602e11  -1.04412e11   2.5217e10    6.37187e9    166.422   166.422    3.39486
   3 │ Earth      0.0     0.0       283.861   0.0166865   1.49598e11  174.